In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import MeCab
from gensim import corpora 
from gensim.models import word2vec

from gensim.models.word2vec import Word2Vec

model_path = 'word2vec.gensim.model'
model = Word2Vec.load(model_path)

mecab = MeCab.Tagger("-Ochasen")

mecab.parse('')

def dist_kyoten(name,html):
    r=requests.get(html)
    soup=BeautifulSoup(r.content,'html.parser')
    l=[]
    for i in soup.find_all():

        if i.string is not None and i.name not in ('script'):

            node = mecab.parseToNode(i.string)

            while node:
                if node.feature.split(u",")[0] == u"名詞" and node.feature.split(u",")[1] in (u"固有名詞", u"一般"):
                    if node.surface in model.wv.vocab:
                        l.append([name,i.name,node.surface,model.similarity(node.surface,'拠点')])
                node = node.next
    return l

df1=pd.DataFrame(dist_kyoten('●●自動車','http://～'))

df.columns = ['会社名', 'タグ', 'ワード','拠点類似度']
df

df.groupby('タグ')['拠点類似度'].max()